In [6]:
import csv
import os
import numpy as np
from sklearn import tree
from sklearn.tree import export_text

root_data_dir = "raw_data_2023_0718"
header = []

def get_avg(path):
    with open(path, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        # 跳过表头
        global header 
        header = next(reader)[1:]
        
        # 跳过时间戳
        data = []
        for row in reader:
            # 跳过含有空值的行
            if all(cell.strip() != '' for cell in row):
                # 去除时间戳
                data.append([float(e) for e in row[1:]])
                
    column_means = np.mean(np.array(data), axis=0)
    return column_means

def normalize(matrix):
    matrix_std = np.std(matrix, axis=0)
    matrix_mean = np.mean(matrix, axis=0)

    zero_std_cols = np.where(matrix_std == 0)[0]

    matrix_mean[zero_std_cols] = 0
    matrix_std[zero_std_cols] = 1

    return (matrix - matrix_mean) / matrix_std

In [7]:
app_data_dirs = [os.path.join(root_data_dir, dir_path) for dir_path in os.listdir(root_data_dir)]

X = []
Y = []
app_names = []
for data_dir in app_data_dirs:
    if not os.path.isdir(data_dir):
        continue
    for file in os.listdir(data_dir):
        file_path = os.path.join(data_dir, file)
        if not os.path.isfile(file_path):
            continue
        X.append(get_avg(file_path))
        Y.append(len(app_names))
        
    app_names.append(os.path.basename(data_dir))

# no need for dts
# X = normalize(X)

['2023-07-18 18:57:40', '0.826', '0.176', '2400', '3.40', '430749', '1024625', '6726', '6733', '0', '0.176', '0.826', '1.66', '286201802', '0.770', '24319', '0.401', '5.54', '424', '992', '3351', '3395', '1028096', '15398912', '1671', '60']
['2023-07-18 18:57:42', '0.834', '0.166', '2400', '3.40', '447553', '1070609', '6967', '6972', '0', '0.166', '0.834', '1.66', '284146997', '0.780', '24169', '0.374', '5.54', '437', '1016', '3462', '3518', '905216', '15247360', '1663', '46.5']
['2023-07-18 18:57:44', '0.832', '0.168', '2400', '3.40', '456883', '1097765', '7391', '7393', '0', '0.168', '0.832', '1.66', '286241661', '0.770', '24361', '0.363', '5.54', '428', '1025', '3382', '3448', '749568', '14954496', '1677', '41']
['2023-07-18 18:57:46', '0.853', '0.146', '2400', '3.40', '431726', '1038060', '6463', '6476', '0', '0.146', '0.853', '1.66', '286809273', '0.775', '24362', '0.404', '5.54', '436', '1013', '3449', '3512', '872448', '15489024', '1673', '49.5']
['2023-07-18 18:57:48', '0.816',

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (146,) + inhomogeneous part.

In [3]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)

r = export_text(clf, feature_names=header, class_names=app_names)
print(r)

|--- vm_cpu_ips <= 139405183.75
|   |--- class: redis
|--- vm_cpu_ips >  139405183.75
|   |--- class: mysql

